In [1]:
!pip install mcp google-adk[a2a] uvicorn httpx python-dotenv --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 2.4 MB/s eta 0:00:00


In [2]:
# Database Setup
# =============================================================================

import sqlite3
from datetime import datetime

DB_PATH = "support.db"

def setup_database():
    """Initialize database with tables and sample data."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Create tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT,
            phone TEXT,
            status TEXT NOT NULL DEFAULT 'active',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS tickets (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_id INTEGER NOT NULL,
            issue TEXT NOT NULL,
            status TEXT NOT NULL DEFAULT 'open',
            priority TEXT NOT NULL DEFAULT 'medium',
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        )
    """)

    # Check if data exists
    cursor.execute("SELECT COUNT(*) FROM customers")
    if cursor.fetchone()[0] == 0:
        # Insert sample customers
        customers = [
            ("John Doe", "john.doe@example.com", "+1-555-0101", "active"),
            ("Jane Smith", "jane.smith@example.com", "+1-555-0102", "active"),
            ("Bob Johnson", "bob.johnson@example.com", "+1-555-0103", "disabled"),
            ("Alice Williams", "alice.w@techcorp.com", "+1-555-0104", "active"),
            ("Charlie Brown", "charlie.brown@email.com", "+1-555-0105", "active"),
        ]
        cursor.executemany(
            "INSERT INTO customers (name, email, phone, status) VALUES (?, ?, ?, ?)",
            customers
        )

        # Insert sample tickets
        tickets = [
            (1, "Cannot login to account", "open", "high"),
            (1, "Password reset not working", "in_progress", "medium"),
            (2, "Billing question", "resolved", "low"),
            (4, "Database timeout errors", "in_progress", "high"),
            (5, "Feature request: dark mode", "open", "low"),
        ]
        cursor.executemany(
            "INSERT INTO tickets (customer_id, issue, status, priority) VALUES (?, ?, ?, ?)",
            tickets
        )
        conn.commit()
        print("Database created with sample data!")
    else:
        print("Database already exists!")

    conn.close()

setup_database()

Database created with sample data!


In [5]:
# MCP Server Implementation
# =============================================================================

from mcp.server.fastmcp import FastMCP
import json

# Create MCP Server
mcp_server = FastMCP(
    name="CustomerSupportMCP",
)

def get_db():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

@mcp_server.tool()
def get_customer(customer_id: int) -> str:
    """Get customer information by ID."""
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers WHERE id = ?", (customer_id,))
    row = cursor.fetchone()
    conn.close()

    if row:
        return json.dumps({"success": True, "customer": dict(row)}, default=str)
    return json.dumps({"success": False, "error": f"Customer {customer_id} not found"})

@mcp_server.tool()
def list_customers(status: str = None, limit: int = 10) -> str:
    """List customers with optional status filter."""
    conn = get_db()
    cursor = conn.cursor()

    if status:
        cursor.execute("SELECT * FROM customers WHERE status = ? LIMIT ?", (status, limit))
    else:
        cursor.execute("SELECT * FROM customers LIMIT ?", (limit,))

    rows = cursor.fetchall()
    conn.close()
    return json.dumps({"success": True, "customers": [dict(r) for r in rows]}, default=str)

@mcp_server.tool()
def update_customer(customer_id: int, email: str = None, phone: str = None) -> str:
    """Update customer information."""
    conn = get_db()
    cursor = conn.cursor()

    updates = []
    values = []
    if email:
        updates.append("email = ?")
        values.append(email)
    if phone:
        updates.append("phone = ?")
        values.append(phone)

    if not updates:
        return json.dumps({"success": False, "error": "No fields to update"})

    values.append(customer_id)
    cursor.execute(f"UPDATE customers SET {', '.join(updates)} WHERE id = ?", values)
    conn.commit()
    conn.close()

    return get_customer(customer_id)

@mcp_server.tool()
def create_ticket(customer_id: int, issue: str, priority: str = "medium") -> str:
    """Create a new support ticket."""
    conn = get_db()
    cursor = conn.cursor()

    cursor.execute(
        "INSERT INTO tickets (customer_id, issue, status, priority) VALUES (?, ?, 'open', ?)",
        (customer_id, issue, priority)
    )
    ticket_id = cursor.lastrowid
    conn.commit()

    cursor.execute("SELECT * FROM tickets WHERE id = ?", (ticket_id,))
    ticket = cursor.fetchone()
    conn.close()

    return json.dumps({"success": True, "ticket": dict(ticket)}, default=str)

@mcp_server.tool()
def get_customer_history(customer_id: int) -> str:
    """Get ticket history for a customer."""
    conn = get_db()
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM customers WHERE id = ?", (customer_id,))
    customer = cursor.fetchone()
    if not customer:
        conn.close()
        return json.dumps({"success": False, "error": "Customer not found"})

    cursor.execute("SELECT * FROM tickets WHERE customer_id = ?", (customer_id,))
    tickets = [dict(r) for r in cursor.fetchall()]
    conn.close()

    return json.dumps({
        "success": True,
        "customer": dict(customer),
        "tickets": tickets,
        "total_tickets": len(tickets)
    }, default=str)

@mcp_server.tool()
def get_active_customers_with_open_tickets() -> str:
    """Get all active customers who have open tickets."""
    conn = get_db()
    cursor = conn.cursor()

    cursor.execute("""
        SELECT DISTINCT c.*, COUNT(t.id) as open_ticket_count
        FROM customers c
        JOIN tickets t ON c.id = t.customer_id
        WHERE c.status = 'active' AND t.status = 'open'
        GROUP BY c.id
    """)

    customers = [dict(r) for r in cursor.fetchall()]
    conn.close()

    return json.dumps({
        "success": True,
        "customers": customers,
        "total": len(customers)
    }, default=str)

print("MCP Server defined with tools:")
print("   - get_customer(customer_id)")
print("   - list_customers(status, limit)")
print("   - update_customer(customer_id, email, phone)")
print("   - create_ticket(customer_id, issue, priority)")
print("   - get_customer_history(customer_id)")
print("   - get_active_customers_with_open_tickets()")

MCP Server defined with tools:
   - get_customer(customer_id)
   - list_customers(status, limit)
   - update_customer(customer_id, email, phone)
   - create_ticket(customer_id, issue, priority)
   - get_customer_history(customer_id)
   - get_active_customers_with_open_tickets()


In [6]:
# A2A Agent Definitions using Google ADK
# =============================================================================

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# ==================== Customer Data Agent ====================

def data_get_customer(customer_id: int) -> dict:
    """Get customer information via MCP."""
    result = json.loads(get_customer(customer_id))
    return result

def data_list_customers(status: str = None, limit: int = 10) -> dict:
    """List customers via MCP."""
    result = json.loads(list_customers(status, limit))
    return result

def data_update_customer(customer_id: int, email: str = None, phone: str = None) -> dict:
    """Update customer via MCP."""
    result = json.loads(update_customer(customer_id, email, phone))
    return result

def data_get_history(customer_id: int) -> dict:
    """Get customer ticket history via MCP."""
    result = json.loads(get_customer_history(customer_id))
    return result

def data_create_ticket(customer_id: int, issue: str, priority: str = "medium") -> dict:
    """Create ticket via MCP."""
    result = json.loads(create_ticket(customer_id, issue, priority))
    return result

def data_get_report() -> dict:
    """Get active customers with open tickets via MCP."""
    result = json.loads(get_active_customers_with_open_tickets())
    return result

# Create Customer Data Agent
customer_data_agent = Agent(
    name="customer_data_agent",
    model="gemini-2.0-flash",
    description="Accesses customer database via MCP tools",
    instruction="""You are the Customer Data Agent. You access the customer database
    via MCP tools. Use the appropriate tool for each request:
    - data_get_customer: Get customer by ID
    - data_list_customers: List customers
    - data_update_customer: Update customer info
    - data_get_history: Get customer's ticket history
    - data_create_ticket: Create new support ticket
    - data_get_report: Get active customers with open tickets
    """,
    tools=[data_get_customer, data_list_customers, data_update_customer,
           data_get_history, data_create_ticket, data_get_report]
)

print("Customer Data Agent defined!")

Customer Data Agent defined!


In [7]:
# ==================== Support Agent ====================

def handle_billing_issue(issue: str) -> dict:
    """Handle billing-related issues."""
    is_urgent = any(w in issue.lower() for w in ["charged twice", "refund", "fraud"])
    return {
        "type": "billing",
        "is_urgent": is_urgent,
        "message": "I understand your billing concern.",
        "resolution_time": "4-8 hours" if is_urgent else "24-48 hours",
        "escalated": is_urgent
    }

def handle_cancellation() -> dict:
    """Handle cancellation requests."""
    return {
        "type": "cancellation",
        "message": "I'm sorry to hear you want to cancel.",
        "retention_offers": ["30-day pause", "Discounted plan"],
        "requires_confirmation": True
    }

def handle_upgrade() -> dict:
    """Handle upgrade requests."""
    return {
        "type": "upgrade",
        "message": "I can help you upgrade!",
        "options": [
            {"tier": "Premium", "price": "$19.99/mo"},
            {"tier": "Enterprise", "price": "$49.99/mo"}
        ],
        "promotion": "20% off first 3 months!"
    }

def handle_urgent(issue: str) -> dict:
    """Handle urgent issues."""
    return {
        "type": "urgent",
        "priority": "HIGH",
        "message": "Escalating immediately!",
        "actions": ["Logged as HIGH priority", "Senior team notified", "Callback in 2 hours"]
    }

# Create Support Agent
support_agent = Agent(
    name="support_agent",
    model="gemini-2.0-flash",
    description="Handles customer support: billing, cancellations, upgrades, urgent issues",
    instruction="""You are the Support Agent. Handle customer support requests:
    - handle_billing_issue: For billing/payment issues
    - handle_cancellation: For cancellation requests
    - handle_upgrade: For upgrade requests
    - handle_urgent: For urgent/emergency issues

    Be empathetic and professional. Escalate urgent issues immediately.
    """,
    tools=[handle_billing_issue, handle_cancellation, handle_upgrade, handle_urgent]
)

print("Support Agent defined!")

Support Agent defined!


In [8]:
# ==================== Router Agent ====================

import re

def analyze_intent(query: str) -> dict:
    """Analyze query to determine intent and routing."""
    query_lower = query.lower()
    intents = []
    params = {}

    # Detect intents
    if any(w in query_lower for w in ["get customer", "customer id", "customer info"]):
        intents.append("get_customer_info")
    if any(w in query_lower for w in ["ticket history", "my tickets"]):
        intents.append("view_history")
    if any(w in query_lower for w in ["update", "change email"]):
        intents.append("update_customer")
    if any(w in query_lower for w in ["billing", "charge", "refund"]):
        intents.append("billing")
    if any(w in query_lower for w in ["cancel"]):
        intents.append("cancellation")
    if any(w in query_lower for w in ["upgrade"]):
        intents.append("upgrade")
    if any(w in query_lower for w in ["urgent", "immediately", "asap"]):
        intents.append("urgent")
    if any(w in query_lower for w in ["all active", "report", "open tickets"]):
        intents.append("report")

    # Extract customer ID
    id_match = re.search(r'(?:customer\s*(?:id)?|id)\s*(\d+)', query_lower)
    if id_match:
        params["customer_id"] = int(id_match.group(1))

    # Extract email
    email_match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', query)
    if email_match:
        params["email"] = email_match.group()

    if not intents:
        intents.append("general_support")

    return {
        "query": query,
        "intents": intents,
        "params": params,
        "is_multi_intent": len(intents) > 1
    }

# Create Router Agent
router_agent = Agent(
    name="router_agent",
    model="gemini-2.0-flash",
    description="Orchestrator that analyzes queries and routes to specialist agents",
    instruction="""You are the Router Agent (Orchestrator).

    1. First use analyze_intent() to understand the query
    2. Based on intents, delegate to appropriate sub-agents:
       - For data operations: Use customer_data_agent
       - For support operations: Use support_agent
    3. For multi-intent queries, coordinate both agents
    4. Synthesize results into a clear response

    Always explain what you're doing and which agent is handling each part.
    """,
    tools=[analyze_intent],
    sub_agents=[customer_data_agent, support_agent]
)

print("Router Agent defined!")

Router Agent defined!


In [9]:
# A2A Agent Card Generation
# =============================================================================

def generate_agent_card(agent, port):
    """Generate A2A-compliant agent card."""
    skills = []
    for tool in agent.tools:
        skills.append({
            "id": tool.__name__,
            "name": tool.__name__.replace("_", " ").title(),
            "description": tool.__doc__ or "No description"
        })

    return {
        "name": agent.name,
        "description": agent.description,
        "version": "1.0.0",
        "url": f"http://localhost:{port}",
        "capabilities": {
            "streaming": False,
            "pushNotifications": False
        },
        "skills": skills,
        "defaultInputModes": ["text"],
        "defaultOutputModes": ["text"]
    }

print("\n" + "="*60)
print("A2A AGENT CARDS")
print("="*60)

print("\n📋 Customer Data Agent Card:")
print(json.dumps(generate_agent_card(customer_data_agent, 8001), indent=2))

print("\n📋 Support Agent Card:")
print(json.dumps(generate_agent_card(support_agent, 8002), indent=2))

print("\n📋 Router Agent Card:")
router_card = generate_agent_card(router_agent, 8003)
router_card["subordinateAgents"] = [
    {"name": "customer_data_agent", "url": "http://localhost:8001"},
    {"name": "support_agent", "url": "http://localhost:8002"}
]
print(json.dumps(router_card, indent=2))


A2A AGENT CARDS

📋 Customer Data Agent Card:
{
  "name": "customer_data_agent",
  "description": "Accesses customer database via MCP tools",
  "version": "1.0.0",
  "url": "http://localhost:8001",
  "capabilities": {
    "streaming": false,
    "pushNotifications": false
  },
  "skills": [
    {
      "id": "data_get_customer",
      "name": "Data Get Customer",
      "description": "Get customer information via MCP."
    },
    {
      "id": "data_list_customers",
      "name": "Data List Customers",
      "description": "List customers via MCP."
    },
    {
      "id": "data_update_customer",
      "name": "Data Update Customer",
      "description": "Update customer via MCP."
    },
    {
      "id": "data_get_history",
      "name": "Data Get History",
      "description": "Get customer ticket history via MCP."
    },
    {
      "id": "data_create_ticket",
      "name": "Data Create Ticket",
      "description": "Create ticket via MCP."
    },
    {
      "id": "data_get_report"

In [10]:
# Run Test Scenarios
# =============================================================================

import asyncio

async def run_agent(agent, query):
    """Run an agent with a query."""
    session_service = InMemorySessionService()
    runner = Runner(agent=agent, app_name="test", session_service=session_service)

    session = await session_service.create_session(app_name="test", user_id="user1")

    response = await runner.run(
        user_id="user1",
        session_id=session.id,
        new_message=query
    )

    return response

def run_test(agent, query):
    """Synchronous wrapper for running agent."""
    return asyncio.get_event_loop().run_until_complete(run_agent(agent, query))

print("\n" + "="*70)
print("RUNNING TEST SCENARIOS")
print("="*70)

test_scenarios = [
    ("Scenario 1: Simple Query (Task Allocation)",
     "Get customer information for ID 5",
     "Single agent MCP call"),

    ("Scenario 2: Coordinated Query",
     "I'm customer 1 and need help upgrading my account",
     "Multiple agents: data fetch + support"),

    ("Scenario 3: Complex Query (Report)",
     "Show me all active customers who have open tickets",
     "Data agent generates report"),

    ("Scenario 4: Escalation (Urgent)",
     "I've been charged twice, please refund immediately!",
     "Support agent handles urgent billing"),

    ("Scenario 5: Multi-Intent",
     "Update my email to new@email.com and show ticket history for customer ID 2",
     "Parallel task execution")
]

for i, (name, query, expected) in enumerate(test_scenarios, 1):
    print(f"\n{'#'*60}")
    print(f"# TEST {i}: {name}")
    print(f"{'#'*60}")
    print(f"Query: \"{query}\"")
    print(f"Expected: {expected}")
    print("-" * 60)

    # Analyze intent first
    intent_result = analyze_intent(query)
    print(f"\n🎯 Intent Analysis:")
    print(f"   Intents: {intent_result['intents']}")
    print(f"   Params: {intent_result['params']}")

    # Run through router agent
    try:
        response = run_test(router_agent, query)
        print(f"\n📋 Response:")
        if hasattr(response, 'content'):
            print(response.content[:500] if len(str(response.content)) > 500 else response.content)
        else:
            print(response)
    except Exception as e:
        print(f"Error: {e}")

    print("\n" + "="*60)


RUNNING TEST SCENARIOS

############################################################
# TEST 1: Scenario 1: Simple Query (Task Allocation)
############################################################
Query: "Get customer information for ID 5"
Expected: Single agent MCP call
------------------------------------------------------------

🎯 Intent Analysis:
   Intents: ['get_customer_info']
   Params: {'customer_id': 5}
Error: This event loop is already running


############################################################
# TEST 2: Scenario 2: Coordinated Query
############################################################
Query: "I'm customer 1 and need help upgrading my account"
Expected: Multiple agents: data fetch + support
------------------------------------------------------------

🎯 Intent Analysis:
   Intents: ['general_support']
   Params: {'customer_id': 1}
Error: This event loop is already running


############################################################
# TEST 3: Scenario 

/tmp/ipython-input-2928437413.py:74: RuntimeWarning: coroutine 'run_agent' was never awaited
  print(f"Error: {e}")


                              CONCLUSION      

What I Learned:
---------------
This assignment taught me how to design and implement a multi-agent system using
two key protocols: Agent-to-Agent (A2A) for inter-agent communication and Model
Context Protocol (MCP) for standardized database access.

The A2A protocol provides a structured way for agents to communicate through
messages and tasks. Each agent has a specific role - the Router Agent acts as
an orchestrator that analyzes user intents and routes requests to specialist
agents (Customer Data Agent and Support Agent). The message-passing pattern
allows for clear tracking of agent interactions and enables complex coordination
scenarios like multi-intent queries and escalations.

The MCP pattern standardizes how agents access external tools and data. By
wrapping database operations in MCP tools (get_customer, create_ticket, etc.),
the system becomes modular and extensible. New tools can be added without
changing the agent logic.

Challenges Faced:
-----------------
1. Intent Detection: Creating robust regex patterns to detect multiple intents
   in a single query was challenging. The solution involved pattern matching
   with careful ordering to avoid false positives.

2. State Management: Passing customer context between agents required careful
   design of the A2A message structure to maintain state across interactions.

3. Coordination Logic: Determining when to involve multiple agents versus
   handling requests with a single agent required careful analysis of query
   complexity and dependencies between data and support functions.

4. Error Handling: Implementing graceful error handling throughout the agent
   pipeline, especially for database errors and invalid parameters.

The final system successfully demonstrates all required scenarios: task
allocation, negotiation/escalation, and multi-step coordination, showing how
A2A and MCP can work together to build practical customer service automation.